# pipeline de análisis de datos para reportes fiscales y contables

## Enfoque: 
Base de Datos funcional y actualizable con Visual Studio Code, MySQL y DBeaver


## Etapas o Prioridades: 
*	Contable: Flujo de Caja
*	Fiscal: Informe periodico de IVA
*	Fiscal: Informe periodico de ICA
*	Fiscal: Informe periodico de Retenciones en la fuente
*	Fiscal: Informe periodico de Impuesto sobre la Renta
*	Fiscal: reporte de informaciòn exógena
*	Contable: Estados Financieros básicos
*	Contable: Notas a Estados Financieros

## OBJETIVO
Crear y actualizar una base de datos en Google Drive u otro lugar en la nube conformada por una tabla de transacciones para generar reportes y archivos de Excel. Esto en un entorno de Visual Studio Code con las extensiones de Jupyter Notebook, Github Copilot, Python y MySQL. Se usará DBeaver como complemento.

## Tablas
La tabla de transacciones junto con las tablas maestras:
* Puede servir de fuente para crear una nueva tabla segmentada que será complementada con información de la correspondiente tabla maestra, con lo cual se crearán muchas tablas.
  * Pros: Distribución de peso de la Base de dato (tablas más pequeñas)
  * Contras: Riesgo de que se pierda o se deje de actualizar una o más tablas. Implementar validaciones constantes
  * Por definir: portabilidad
* Puede ser actualizada con la información de las tablas maestras. Un campo para actualizar cada tabla maestra, con lo cual se generará una sola tabla (robusta y/o pesada).
  * Pro: Una sola tabla que facilita el control y las validaciones
  * Pro: Integridad, elimina el riesgo de se olvide trasladar o actualizar una tabla
  * Contras: Puede llegar a ser muy pesada
  * Por definir: portabilidad

## JUSTIFICACIÓN
1.	Construir proyectos que se visualicen en Github usando herramientas de análisis de datos y manejo de bases de datos.
2.	Desarrollar una herramienta de automatice el trabajo.
3.	Estructurar la base de datos para el desarrollo de aplicación web o escritorio.
4.	Adquirir experiencia

## HERRAMIENTAS
* Entorno virtual.
* Visual Studio Code + extensiones1 → Jupyter Notebook, Python, MySQL y Github Copilot.
* Git y Github como herramienta de versionamiento.
* Dbeaver.

Entorno virtual
* Abrir la terminal integrada en Visual Studio Code (View -> Terminal)
* cd path_to_your_environment → ruta al directorio donde se encuentra tu entorno virtual
* \env\Scripts\activate → activa el entorno virtual

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
# Cambia el directorio de trabajo a 'C:/path/to/your/directory'
os.chdir('C:/Emprendimiento/Pry_Datos/')

# Imprime la ruta del directorio de trabajo actual
print(os.getcwd())

C:\Emprendimiento\Pry_Datos


In [3]:
#import sys
#!{sys.executable} -m pip install xlrd

# Lectura de archivo xls: instalar paquete xlrd
df_Bce_00 = pd.read_excel(r'data_2023_Bce.xls')

# Muestra las primeras 5 filas del DataFrame
print(df_Bce_00.head())


   CUENTA  NIT / C. C.  TIPO             NOMBRE  NIVEL NATURALEZA  \
0       1          NaN   NaN             ACTIVO      1          D   
1      11          NaN   NaN         DISPONIBLE      2          D   
2    1105          NaN   NaN               CAJA      3          D   
3  110505          NaN   NaN       CAJA GENERAL      4          D   
4    1120          NaN   NaN  CUENTAS DE AHORRO      3          D   

   SALDO INICIAL        DEBITO       CREDITO   SALDO FINAL  
0   2.807411e+08  2.014495e+09  2.126990e+09  1.682466e+08  
1   3.964723e+07  8.628593e+08  8.537267e+08  4.877987e+07  
2   3.800000e+01  0.000000e+00  3.800000e+01  0.000000e+00  
3   3.800000e+01  0.000000e+00  3.800000e+01  0.000000e+00  
4   3.964720e+07  8.628593e+08  8.537267e+08  4.877987e+07  


In [4]:
print(df_Bce_00.columns)
df_Bce_00.info()

Index(['CUENTA', 'NIT / C. C.', 'TIPO', 'NOMBRE', 'NIVEL', 'NATURALEZA',
       'SALDO INICIAL', 'DEBITO', 'CREDITO', 'SALDO FINAL'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CUENTA         262 non-null    int64  
 1   NIT / C. C.    0 non-null      float64
 2   TIPO           0 non-null      float64
 3   NOMBRE         262 non-null    object 
 4   NIVEL          262 non-null    int64  
 5   NATURALEZA     262 non-null    object 
 6   SALDO INICIAL  262 non-null    float64
 7   DEBITO         262 non-null    float64
 8   CREDITO        262 non-null    float64
 9   SALDO FINAL    262 non-null    float64
dtypes: float64(6), int64(2), object(2)
memory usage: 20.6+ KB


In [5]:
df_Bce_00.drop('TIPO', axis=1, inplace=True)
df_Bce_00.drop('NIT / C. C.', axis=1, inplace=True)
df_Bce_00.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CUENTA         262 non-null    int64  
 1   NOMBRE         262 non-null    object 
 2   NIVEL          262 non-null    int64  
 3   NATURALEZA     262 non-null    object 
 4   SALDO INICIAL  262 non-null    float64
 5   DEBITO         262 non-null    float64
 6   CREDITO        262 non-null    float64
 7   SALDO FINAL    262 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 16.5+ KB


In [6]:
#import sys
# !{sys.executable} -m pip install openpyxl

# Lectura de archivo xlsx: previamente instalar paquete openpyxl 
df_Mov_00 = pd.read_excel(r'data_2023_Mov.xlsx')

# Muestra las primeras 5 filas del DataFrame
df_Mov_00.head()

,FECHA,DOCUMENTO,TIPODOC,NUMDOC,CUENTA,NOM_CUENTA,CONCEPTO,NATURALEZA,CENTRO,CODIGO_CTABANCARIA,...,IND_CONTABILIDAD,DV,NOMBRETERCERO,CUENTA_BANCARIA,NOM_CENTRO,DESCRIPCION_CORTA,DIRECCION,TELEFONOS,NOM_CIUDAD,C_C
0,2023/04/19,FC 00000433,FC,433,51350501,SERVICIO ASEO,OPERARIO DE ASEO Y LIMPIEZA,D,NaN,NaN,...,11263,3.0,FIXXTER SAS,NaN,NaN,NaN,CL 126 70 G 62,2853984,BOGOTÁ,NaN
1,2023/04/19,FC 00000433,FC,433,51350501,SERVICIO ASEO,OPERARIO DE ASEO Y LIMPIEZA,D,NaN,NaN,...,11264,3.0,FIXXTER SAS,NaN,NaN,NaN,CL 126 70 G 62,2853984,BOGOTÁ,NaN
2,2023/04/19,FC 00000433,FC,433,24081002,IVA DESCONTABLE SERVICIOS 19% TRANSITORIO,OPERARIO DE ASEO Y LIMPIEZA,D,NaN,NaN,...,11265,3.0,FIXXTER SAS,NaN,NaN,NaN,CL 126 70 G 62,2853984,BOGOTÁ,NaN
3,2023/04/19,FC 00000433,FC,433,23355004,SERVICIO DE ASEO,OPERARIO DE ASEO Y LIMPIEZA,C,NaN,NaN,...,11266,3.0,FIXXTER SAS,NaN,NaN,NaN,CL 126 70 G 62,2853984,BOGOTÁ,NaN
4,2023/01/01,CC 00000073,CC,73,361005,PERDIDA DEL EJERCICIO,TRASLADO DE RESULTADOS DE AÑO 2022,C,NaN,NaN,...,6124,1.0,CO SERVICES S.A.S,NaN,NaN,NaN,CR 20 187 71 SEC 13 OF 47,3144307737,BOGOTÁ,NaN


In [7]:
# Elimina las columnas 'NATURALEZA' y 'CENTRO'
df_Mov_00 = df_Mov_00.drop(['NOM_CUENTA', 'NATURALEZA', 'CENTRO', 'CODIGO_CTABANCARIA', 'CODIGO_USUARIO', 'FECHA_SISTEMA', 'IND_CONTABILIDAD', 'CUENTA_BANCARIA', 'NOM_CENTRO', 'DESCRIPCION_CORTA', 'DIRECCION', 'TELEFONOS', 'NOM_CIUDAD', 'C_C'], axis=1)

# Muestra las primeras 5 filas del DataFrame
df_Mov_00.head()

,FECHA,DOCUMENTO,TIPODOC,NUMDOC,CUENTA,CONCEPTO,DEBITO,CREDITO,IDENTIDADTERCERO,DOC_FUENTE,DV,NOMBRETERCERO
0,2023/04/19,FC 00000433,FC,433,51350501,OPERARIO DE ASEO Y LIMPIEZA,2280607.68,0.00,900994552,FEV-2023,3.0,FIXXTER SAS
1,2023/04/19,FC 00000433,FC,433,51350501,OPERARIO DE ASEO Y LIMPIEZA,228060.77,0.00,900994552,FEV-2023,3.0,FIXXTER SAS
2,2023/04/19,FC 00000433,FC,433,24081002,OPERARIO DE ASEO Y LIMPIEZA,43331.55,0.00,900994552,FEV-2023,3.0,FIXXTER SAS
3,2023/04/19,FC 00000433,FC,433,23355004,OPERARIO DE ASEO Y LIMPIEZA,0.00,2552000.00,900994552,FEV-2023,3.0,FIXXTER SAS
4,2023/01/01,CC 00000073,CC,73,361005,TRASLADO DE RESULTADOS DE AÑO 2022,0.00,71627508.55,901513634,CC 00000073,1.0,CO SERVICES S.A.S


In [8]:
df_Mov_00.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3868 entries, 0 to 3867
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   FECHA             3868 non-null   object 
 1   DOCUMENTO         3868 non-null   object 
 2   TIPODOC           3868 non-null   object 
 3   NUMDOC            3868 non-null   int64  
 4   CUENTA            3868 non-null   int64  
 5   CONCEPTO          3868 non-null   object 
 6   DEBITO            3868 non-null   float64
 7   CREDITO           3868 non-null   float64
 8   IDENTIDADTERCERO  3868 non-null   int64  
 9   DOC_FUENTE        2812 non-null   object 
 10  DV                2268 non-null   float64
 11  NOMBRETERCERO     3868 non-null   object 
dtypes: float64(3), int64(3), object(6)
memory usage: 362.8+ KB


In [9]:
df_Mov_00['FECHA'] = pd.to_datetime(df_Mov_00['FECHA'], format='%Y/%m/%d')
df_Mov_00.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3868 entries, 0 to 3867
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   FECHA             3868 non-null   datetime64[ns]
 1   DOCUMENTO         3868 non-null   object        
 2   TIPODOC           3868 non-null   object        
 3   NUMDOC            3868 non-null   int64         
 4   CUENTA            3868 non-null   int64         
 5   CONCEPTO          3868 non-null   object        
 6   DEBITO            3868 non-null   float64       
 7   CREDITO           3868 non-null   float64       
 8   IDENTIDADTERCERO  3868 non-null   int64         
 9   DOC_FUENTE        2812 non-null   object        
 10  DV                2268 non-null   float64       
 11  NOMBRETERCERO     3868 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 362.8+ KB


In [10]:
# Los datos de la columna DV no requieren ser convertidos a numéricos porque no se realizarán operaciones aritméticas con ellos
df_Mov_00['DV'] = df_Mov_00['DV'].astype('object')
df_Mov_00.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3868 entries, 0 to 3867
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   FECHA             3868 non-null   datetime64[ns]
 1   DOCUMENTO         3868 non-null   object        
 2   TIPODOC           3868 non-null   object        
 3   NUMDOC            3868 non-null   int64         
 4   CUENTA            3868 non-null   int64         
 5   CONCEPTO          3868 non-null   object        
 6   DEBITO            3868 non-null   float64       
 7   CREDITO           3868 non-null   float64       
 8   IDENTIDADTERCERO  3868 non-null   int64         
 9   DOC_FUENTE        2812 non-null   object        
 10  DV                2268 non-null   object        
 11  NOMBRETERCERO     3868 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 362.8+ KB


In [11]:
# Obtiene la posición de la columna 'DEBITO'
col_index = df_Mov_00.columns.get_loc('DEBITO')

# Inserta la nueva columna en la posición correcta
df_Mov_00.insert(col_index, 'NETO', df_Mov_00['DEBITO'] - df_Mov_00['CREDITO'])

df_Mov_00.head()


,FECHA,DOCUMENTO,TIPODOC,NUMDOC,CUENTA,CONCEPTO,NETO,DEBITO,CREDITO,IDENTIDADTERCERO,DOC_FUENTE,DV,NOMBRETERCERO
0,2023-04-19,FC 00000433,FC,433,51350501,OPERARIO DE ASEO Y LIMPIEZA,2280607.68,2280607.68,0.00,900994552,FEV-2023,3.0,FIXXTER SAS
1,2023-04-19,FC 00000433,FC,433,51350501,OPERARIO DE ASEO Y LIMPIEZA,228060.77,228060.77,0.00,900994552,FEV-2023,3.0,FIXXTER SAS
2,2023-04-19,FC 00000433,FC,433,24081002,OPERARIO DE ASEO Y LIMPIEZA,43331.55,43331.55,0.00,900994552,FEV-2023,3.0,FIXXTER SAS
3,2023-04-19,FC 00000433,FC,433,23355004,OPERARIO DE ASEO Y LIMPIEZA,-2552000.00,0.00,2552000.00,900994552,FEV-2023,3.0,FIXXTER SAS
4,2023-01-01,CC 00000073,CC,73,361005,TRASLADO DE RESULTADOS DE AÑO 2022,-71627508.55,0.00,71627508.55,901513634,CC 00000073,1.0,CO SERVICES S.A.S


In [12]:
# Define una función de formato
def format_with_commas(x):
    return '{:,}'.format(x)

# Aplica la función de formato a las columnas seleccionadas
df_Mov_00[['NETO', 'DEBITO', 'CREDITO']] = df_Mov_00[['NETO', 'DEBITO', 'CREDITO']].applymap(format_with_commas)

df_Mov_00.head()

C:\Users\gusta\AppData\Local\Temp\ipykernel_1944\250674207.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_Mov_00[['NETO', 'DEBITO', 'CREDITO']] = df_Mov_00[['NETO', 'DEBITO', 'CREDITO']].applymap(format_with_commas)


,FECHA,DOCUMENTO,TIPODOC,NUMDOC,CUENTA,CONCEPTO,NETO,DEBITO,CREDITO,IDENTIDADTERCERO,DOC_FUENTE,DV,NOMBRETERCERO
0,2023-04-19,FC 00000433,FC,433,51350501,OPERARIO DE ASEO Y LIMPIEZA,"2,280,607.68","2,280,607.68",0.0,900994552,FEV-2023,3.0,FIXXTER SAS
1,2023-04-19,FC 00000433,FC,433,51350501,OPERARIO DE ASEO Y LIMPIEZA,"228,060.77","228,060.77",0.0,900994552,FEV-2023,3.0,FIXXTER SAS
2,2023-04-19,FC 00000433,FC,433,24081002,OPERARIO DE ASEO Y LIMPIEZA,"43,331.55","43,331.55",0.0,900994552,FEV-2023,3.0,FIXXTER SAS
3,2023-04-19,FC 00000433,FC,433,23355004,OPERARIO DE ASEO Y LIMPIEZA,"-2,552,000.0",0.0,"2,552,000.0",900994552,FEV-2023,3.0,FIXXTER SAS
4,2023-01-01,CC 00000073,CC,73,361005,TRASLADO DE RESULTADOS DE AÑO 2022,"-71,627,508.55",0.0,"71,627,508.55",901513634,CC 00000073,1.0,CO SERVICES S.A.S


In [13]:
df_Mov_00.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3868 entries, 0 to 3867
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   FECHA             3868 non-null   datetime64[ns]
 1   DOCUMENTO         3868 non-null   object        
 2   TIPODOC           3868 non-null   object        
 3   NUMDOC            3868 non-null   int64         
 4   CUENTA            3868 non-null   int64         
 5   CONCEPTO          3868 non-null   object        
 6   NETO              3868 non-null   object        
 7   DEBITO            3868 non-null   object        
 8   CREDITO           3868 non-null   object        
 9   IDENTIDADTERCERO  3868 non-null   int64         
 10  DOC_FUENTE        2812 non-null   object        
 11  DV                2268 non-null   object        
 12  NOMBRETERCERO     3868 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(9)
memory usage: 393.0+ KB


==============================================================================================================================

Bases de Datos
* Crear/cargar códigos de grupos
* Crear/cargar códigos de items
* Crear/cargar calendario con semanas


==============================================================================================================================

Cargar información en Dataframe con registros para Flujo de Caja Directo
* Cargar desde la tabla/Base de datos códigos de item y grupo
* Cargar desde la tabla/Base de datos calendario: semana, mes y año al dataframe
* Advertir sobre los registros que quedan sin código

Dataframe: Función para seleccionar los documentos que se deben seleccionar
* Por tipo de documento → Opción desechada
* Aquellos que tienen algún registro cuyo campo item = "Cash and equivalent"

Base de datos
* Base de datos con datos reales (actualizar: funciones de agrupación o agregación) desde el Dataframe con registros Flujo de Caja Directo
* Base de datos con datos proyectados (cargar cifras)

Reportes
* Flujo de Caja Directo con datos reales
* Flujo de Caja Directo con datos proyectados
* Flujo de Caja Directo con datos reales (ytd) y estimados (forecast)

Convertir a un archivo Python ejecutable

# Primera vez
* git init
* git add .
* git commit -m "Initial commit"
* git remote remove origin (opcional)
* git remote add origin https://github.com/rosillogithub/Flujo_Caja_Directo.git
* git pull origin main --allow-unrelated-histories 
* git pull --rebase origin main (cuando la situación lo requiere)
* git push -u origin main

git remote -v se usa para ver a qué URL apunta tu repositorio Git actualmente. Este comando muestra todas las conexiones remotas que tienes configuradas y las URL a las que apuntan.
* origin  https://github.com/rosillogithub/Flujo_Caja_Directo.git (fetch)
* origin  https://github.com/rosillogithub/Flujo_Caja_Directo.git (push)

fetch: Este comando se utiliza para obtener los últimos cambios desde un repositorio remoto, pero sin fusionarlos con tu rama local. Es decir, git fetch te permite ver los cambios recientes en el repositorio remoto, pero no afecta tu trabajo local. Después de hacer un fetch, puedes decidir si quieres fusionar esos cambios en tu rama local con git merge.

push: Este comando se utiliza para enviar tus cambios locales a un repositorio remoto. Después de hacer cambios en tu repositorio local y hacer commit de esos cambios, puedes usar git push para subir esos cambios al repositorio remoto, para que otras personas puedan verlos y obtenerlos.

==========================================================================================================================================================

git push se utiliza para subir (push) tus cambios locales a un repositorio remoto. Después de hacer cambios en tu repositorio local y hacer commit de esos cambios, puedes usar git push para subir esos cambios al repositorio remoto, para que otras personas puedan verlos y obtenerlos.

git push -f origin main
* Se usa para subir los cambios locales al repositorio remoto sin descargar los cambios del repositorio remoto, puedes usar git push -f o git push --force. 
* Este comando forzará a Git a subir los cambios locales al repositorio remoto, incluso si esto sobrescribe los cambios en el repositorio remoto.

==========================================================================================================================================================

git pull se utiliza para obtener (pull) los cambios más recientes desde un repositorio remoto a tu repositorio local. Si otras personas han realizado cambios en el repositorio remoto y los han subido (push), puedes usar git pull para obtener esos cambios y mantener tu repositorio local actualizado.

Crear el archivo .gitignore
* echo "" > .gitignore

# GitHub
Ahora que has hecho push de tu repositorio local a GitHub, puedes hacer lo siguiente en GitHub:

1. Verifica tus cambios: Ve a la URL de tu repositorio (https://github.com/rosillogithub/Flujo_Caja_Directo.git) y deberías ver todos los archivos y commits que has hecho push desde tu repositorio local.

2. Continúa trabajando en tu proyecto: Puedes seguir haciendo cambios en tu repositorio local y luego hacer push de esos cambios a GitHub con git push.

3. Comparte tu proyecto: Puedes compartir la URL de tu repositorio con otras personas para que puedan ver, clonar o bifurcar tu proyecto.

4. Colabora con otros: Si estás trabajando con otros en este proyecto, pueden clonar el repositorio a sus propias máquinas y luego hacer push de sus cambios a GitHub.

5. Aprovecha las características de GitHub: GitHub tiene muchas características útiles, como la gestión de problemas, las solicitudes de extracción, las páginas de GitHub, y más. Puedes explorar estas características para ver cómo pueden ayudarte en tu proyecto.

Recuerda que cada vez que hagas un cambio en tu repositorio local que quieras subir a GitHub, debes hacer un commit de ese cambio con git commit y luego hacer push del commit a GitHub con git push.



# Siguientes sincronizaciones con GitHub
* git init
* git add .
* git commit -m "descripción"
* git remote -v (opcional): Muestra todas las conexiones remotas que tienes configuradas y las URL a las que apuntan.
* git push -u origin main: Después cambios en el repositorio local, hacer commit de los cambios, "push" sube los cambios al repositorio remoto.
* git push -f origin main (opcional): Sube los cambios locales al repositorio remoto sin descargar los cambios del repositorio remoto (git push -f o git push --force). Este comando forzará a Git a subir los cambios locales al repositorio remoto, incluso si esto sobrescribe los cambios en el repositorio remoto.

==========================================================================================================================================================

git pull se utiliza para obtener (pull) los cambios más recientes desde un repositorio remoto a tu repositorio local. Si otras personas han realizado cambios en el repositorio remoto y los han subido (push), puedes usar git pull para obtener esos cambios y mantener tu repositorio local actualizado.